In [1]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd
from pathlib import Path
import os
from plotnine import *
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from descartes import PolygonPatch
import alphashape
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.geometry import Point
import math
from scipy.stats import kde
import hdbscan
from sklearn.cluster import DBSCAN
from scipy.spatial import Delaunay
from collections import defaultdict
from itertools import permutations
import plotly.figure_factory as ff
import matplotlib.cm as cm
import itertools
from scipy.stats import wilcoxon #Wilcoxon is nonparametric, paired data = paired student T test ***
from scipy.stats import pearsonr

# Number of neighbors in different types of cluster
# Load cluster-participant cell composition
df_Cluster_pv2 = pd.read_csv("C:/Data/DCs_results/c_Type.txt")
df_Cluster_pv2_Pt = df_Cluster_pv2[~df_Cluster_pv2["Pt"].isin(["Norm LN1x4","Norm LN2x4","Norm LN3x4","Norm LN4x4"])].drop(columns=["Unnamed: 0"])
# Load cluster-overall size (include all cells)
SIZE_cType = pd.read_csv("C:/Data/DCs_results/SIZE_c_Type.txt")
SIZE_cType_Pt = SIZE_cType[~SIZE_cType["Pt"].isin(["Norm LN1x4","Norm LN2x4","Norm LN3x4","Norm LN4x4"])].drop(columns=["Unnamed: 0"])
# Load cluster-overall size (include all cells, with mean in each specimen)
SIZE_cType_Pts = pd.read_csv("C:/Data/DCs_results/SIZE_cType_Pts.txt")
# Load neighbor-cluster data, remove control samples (each cell, numbers of neighbors, participated cluster ID)
all_files_cType = pd.read_csv("C:/Data/DCs_results/nn_DCs_clustera_cType.txt")
all_files_cType_Pt = all_files_cType[~all_files_cType["Pt"].isin(["Norm LN1x4","Norm LN2x4","Norm LN3x4","Norm LN4x4"])].drop(columns=["Unnamed: 0", "Unnamed: 0.1"]).dropna(
    subset=["X", "Y"]).fillna(0)

# Hetero-cDC1
neighbors_for_cDC1_Het = all_files_cType_Pt[(all_files_cType_Pt.c_Type == "Het")&(all_files_cType_Pt.Phenotype.isin(["CD141+PD-L1-","CD141+PD-L1+"]))]
neighbors_for_cDC1_Het["With Total CD1a+ Neighbors"] = neighbors_for_cDC1_Het["CD1a+"] + neighbors_for_cDC1_Het["CD1a+PD-L1+"] + neighbors_for_cDC1_Het["CD1a+PD-L1-"]
neighbors_for_cDC1_Het2 = neighbors_for_cDC1_Het[["Pt","Cluster","With Total CD1a+ Neighbors"]] 

# No. of cDC1 having CD1a+ neighbors
neighbors_for_cDC1_Het2_withCD1aNeighbor= neighbors_for_cDC1_Het2[(neighbors_for_cDC1_Het2["With Total CD1a+ Neighbors"]>0)].drop(columns=["With Total CD1a+ Neighbors"])
neighbors_for_cDC1_Het2_withCD1aNeighbor2 = neighbors_for_cDC1_Het2_withCD1aNeighbor.pivot_table(index=["Pt","Cluster"], aggfunc=lambda x:len(x)).to_frame()
neighbors_for_cDC1_Het2_withCD1aNeighbor2.reset_index(inplace=True)
neighbors_for_cDC1_Het2_withCD1aNeighbor3 = neighbors_for_cDC1_Het2_withCD1aNeighbor2.rename(columns={0:"No. cells with CD1a+ neighbors"})
# No. of cDC1 without CD1a+ neighbors
neighbors_for_cDC1_Het2_woCD1aNeighbor= neighbors_for_cDC1_Het2[(neighbors_for_cDC1_Het2["With Total CD1a+ Neighbors"]==0)].drop(columns=["With Total CD1a+ Neighbors"])
neighbors_for_cDC1_Het2_woCD1aNeighbor2 = neighbors_for_cDC1_Het2_woCD1aNeighbor.pivot_table(index=["Pt","Cluster"], aggfunc=lambda x:len(x)).to_frame()
neighbors_for_cDC1_Het2_woCD1aNeighbor2.reset_index(inplace=True)
neighbors_for_cDC1_Het2_woCD1aNeighbor3 = neighbors_for_cDC1_Het2_woCD1aNeighbor2.rename(columns={0:"No. cells wo CD1a+ neighbors"})
# Put the counts together
neighbors_for_cDC1_Het2_all = pd.merge(neighbors_for_cDC1_Het2_withCD1aNeighbor3,neighbors_for_cDC1_Het2_woCD1aNeighbor3, how="outer").fillna(0)
neighbors_for_cDC1_Het2_all["%with CD1a+ on cDC1"] = 100*neighbors_for_cDC1_Het2_all["No. cells with CD1a+ neighbors"]/(
    neighbors_for_cDC1_Het2_all["No. cells with CD1a+ neighbors"] + neighbors_for_cDC1_Het2_all["No. cells wo CD1a+ neighbors"])
neighbors_for_cDC1_Het2_all2 = neighbors_for_cDC1_Het2_all.round(1)[["Pt","Cluster","%with CD1a+ on cDC1"]]
#Size
size = SIZE_cType[(SIZE_cType.c_Type=="Het")][["Pt","Cluster","Sum_all"]]
size_mod = size.replace({"Pt":{"20190222_RZ_DC PD-L1 Panel_12195-09 LNNA1x5":"12195-09 LNNA1x5",
                               "20190222_RZ_DC PD-L1 Panel_12195-13 LNNA1x5":"12195-13 LNNA1x5",
                               "20190222_RZ_DC PD-L1 Panel_12195-23 LNNC2x6":"12195-23 LNNC2x6",
                               "20190222_RZ_DC PD-L1 Panel_14346-1-23 (Block #A2)x4":"14346-1-23 (Block #A2)x4",
                               "20190222_RZ_DC PD-L1 Panel_14346-1-28 (Block #B1)x6":"14346-1-28 (Block #B1)x6",
                               "20190222_RZ_DC PD-L1 Panel_14346-1-37 C1x3":"14346-1-37 C1x3"}})
# Add together
size_percentwithCD1A = pd.merge(size_mod,neighbors_for_cDC1_Het2_all2, on=["Pt","Cluster"], how="outer")
size_percentwithCD1A.to_csv("C:/Data/DCs_results/SIZE_percentWITHcd1aNeighbor_ONcDC1Heters.txt")

# PD-L1 per Hetero
PDL1perHetero = df_Cluster_pv2[(df_Cluster_pv2.c_Type=="Het")]
PDL1perHetero["%PD-L1+ on cDC1"] = 100*PDL1perHetero["CD141+PD-L1+"]/(PDL1perHetero["CD141+PD-L1+"]+PDL1perHetero["CD141+PD-L1-"])
PDL1perHetero2 = PDL1perHetero[["Pt","Cluster","%PD-L1+ on cDC1"]].round(1)
PDL1perHetero3 = PDL1perHetero2.replace({"Pt":{"20190222_RZ_DC PD-L1 Panel_12195-09 LNNA1x5":"12195-09 LNNA1x5",
                               "20190222_RZ_DC PD-L1 Panel_12195-13 LNNA1x5":"12195-13 LNNA1x5",
                               "20190222_RZ_DC PD-L1 Panel_12195-23 LNNC2x6":"12195-23 LNNC2x6",
                               "20190222_RZ_DC PD-L1 Panel_14346-1-23 (Block #A2)x4":"14346-1-23 (Block #A2)x4",
                               "20190222_RZ_DC PD-L1 Panel_14346-1-28 (Block #B1)x6":"14346-1-28 (Block #B1)x6",
                               "20190222_RZ_DC PD-L1 Panel_14346-1-37 C1x3":"14346-1-37 C1x3"}})
PDL1perHetero4 = PDL1perHetero3[~PDL1perHetero3.Pt.isin(["Norm LN1x4","Norm LN2x4","Norm LN3x4","Norm LN4x4"])]

# Put all together 
Size_nn_PDL1_ONheterocDC1 = pd.merge(size_percentwithCD1A, PDL1perHetero4, on=["Pt","Cluster"], how="outer").rename(columns={"Sum_all":"Hetero-Cluster Size(Total Cells)"})
Size_nn_PDL1_ONheterocDC1.to_csv("C:/Data/DCs_results/Size_nn_PDL1_ONheterocDC1.txt")
mean = Size_nn_PDL1_ONheterocDC1.pivot_table(index=["Pt"], values=["Hetero-Cluster Size(Total Cells)",
                                                           "%with CD1a+ on cDC1",
                                                           "%PD-L1+ on cDC1"], aggfunc=lambda x:np.mean(x)).round(1)

# Plot size vs. %PD-L1 on hetero-cDC1
sns.set(style = "ticks", font_scale = 1.5, color_codes=True)
sns.set_context("talk")
g = sns.regplot(data=mean , x="Hetero-Cluster Size(Total Cells)", y="%PD-L1+ on cDC1", scatter_kws={"s": 50, "color":"grey","edgecolor":"black"},
               line_kws={"color":"grey"})

plt.xlabel('Hetero-Cluster Size(Total Cells)', fontsize=17)
plt.ylabel('%PD-L1+ on cDC1',fontsize=17)
plt.tight_layout()
g.figure.savefig('C:/Data/DCs_results/HetSize_vs_%PD-L1ONcDC1heteros.tiff', format='tiff', dpi=600)
plt.show()

# Plot size vs. %wtih CD1a+ neighbors on hetero-cDC1
sns.set(style = "ticks", font_scale = 1.5, color_codes=True)
sns.set_context("talk")
g = sns.regplot(data=mean , x="Hetero-Cluster Size(Total Cells)", y="%with CD1a+ on cDC1", scatter_kws={"s": 50, "color":"grey","edgecolor":"black"},
               line_kws={"color":"grey"})
plt.xlabel('Hetero-Cluster Size(Total Cells)', fontsize=17)
plt.ylabel('%with CD1a+ neighbors',fontsize=17)
plt.tight_layout()
g.figure.savefig('C:/Data/DCs_results/HetSize_vs_%withCD1a+neighborsONheterocDC1.tiff', format='tiff', dpi=600)
plt.show()